In [1]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from sympy import Point, Polygon 
import math
from fastapi import FastAPI, File, UploadFile, Form, HTTPException, Depends, APIRouter
from fastapi.responses import FileResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.security import OAuth2PasswordBearer
import os
import uvicorn
from threading import Thread
from io import BytesIO
from pydantic import BaseModel
from passlib.context import CryptContext
from pymongo import MongoClient
import httpx
import jwt
import secrets
from datetime import datetime, timedelta
from jose import JWTError
from bson import ObjectId
import base64
import configurare
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calcul_unghi_2_drepte(a, b, c, d):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    d = np.array(d)
    
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi



In [3]:
def calcul_unghi_orizont(a, b):
    a = np.array(a)
    b = np.array(b)
    c = [0, 0.5, 0]
    d = [0.5, 0.5, 0]
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi

In [4]:
def prelucrare_img(img, tip):
    global factor_scalare
    global pose_model
    distante_m = []

    
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
    
        #detectarea modelului
        pose_model = pose.process(img)
            
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        factor_scalare = int(img.shape[0] / 720)
        if tip == 1:
            (img, distante_m) = adaug_linii_suport_front(img)
            (img_al, _) = afisare_aliniament1(img, mijloace, 1);
            #print(distante_m)
        else:
            (img, scalare) = adaug_linii_suport_profil(img)
            (img_al, distante_pixeli) = afisare_aliniament1(img,  puncte_de_interes, 2);
            
            for dist in distante_pixeli:
                distante_m.append(round(100 * scalare * dist, 2))
            #print(distante_m_profil)

        copie = img.copy()
        for lin in range(0, img.shape[1], 50 * factor_scalare):
            cv2.line(copie, (lin, 0), (lin, img.shape[0]), (200, 200, 200), 1)

        for lin in range(0, img.shape[0], 50 * factor_scalare):
            cv2.line(copie, (0, lin), (img.shape[1], lin), (200, 200, 200), 1)

        img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)
        img_al = cv2.addWeighted(img_al, 0.8, copie, 0.2, 0)
        cv2.imwrite('img_prelucrata.jpg', img)

        #cv2.imwrite(path_salvare, img_al)

    return distante_m, img_al


In [5]:
def preprocesare_date_img(distante):
    dict_distante = {}

    dict_distante["diferenta_urechi"] = distante["front"][0]
    dict_distante["diferenta_umeri"] = distante["front"][1]
    dict_distante["diferenta_solduri"] = distante["front"][2]
    dict_distante["diferenta_genunchi"] = distante["front"][3]
    dict_distante["diferenta_glezne"] = distante["front"][4]

    dict_distante["umeri"] = distante["profil"][1]
    dict_distante["solduri"] = distante["profil"][2]
    dict_distante["genunchi"] = distante["profil"][3]
    dict_distante["glezne"] = distante["profil"][4]

    print(dict_distante)
    return dict_distante

In [6]:
def setup_api():
    app = FastAPI()

    app.add_middleware(
        CORSMiddleware,
        allow_origins = ["*"],  # Permite cereri din orice sursă
        allow_credentials = True,
        allow_methods = ["*"],  # Permite toate metodele HTTP (GET, POST etc.)
        allow_headers= ["*"],  # Permite toate header-ele
    )
    #adaugate pt ca serverele nu puteau comunica prin live server-ul de la vscode. Poate scot partea asta

    app.mount("/processed", StaticFiles(directory="processed"), name="processed")

    

    return app
    

In [7]:
def get_db():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["aplicatie"]
    colectie_utilizatori = db["utilizatori"]
    colectie_teste_postura = db["teste_postura"]

    return db

In [8]:
def generare_token_acces(utilizator_curent: dict):
    token = utilizator_curent.copy()

    exp = (datetime.utcnow() + timedelta(days = 1)).timestamp()
    token.update({"exp" : exp})
    token_criptat = jwt.encode(token, cheie_autentificare, algorithm = "HS256")

    return token_criptat

In [9]:
app = setup_api()
router = APIRouter()
app.include_router(router)
global distante
global cheie_autentificare
global oauth2

class Utilizator(BaseModel):
    email : str
    parola : str
    username : str

class DateLogare(BaseModel):
    parola : str
    username : str

class DateTestPostura(BaseModel):
    utilizator_id: str
    rezultate: dict
    imagine_front: str
    imagine_profil : str

oauth2 = OAuth2PasswordBearer(tokenUrl = "/logare_cont/")
cheie_autentificare = configurare.CHEIE_AUTENTIF
distante = {"front": None, "profil": None}

@app.get("/utilizator_curent/")
async def get_utilizator_curent(token_criptat: str = Depends(oauth2), db = Depends(get_db)):

    colectie_utilizatori = db["utilizatori"]
    try:
        token_decodat = jwt.decode(token_criptat, cheie_autentificare, algorithms = ["HS256"])
        utilizator_id = ObjectId(token_decodat["utilizator_id"])
        print(utilizator_id)

        if utilizator_id == None:
            raise HTTPException(status_code=401, detail="Tokenul nu a putut fi decodat")
        else:
            utilizator = colectie_utilizatori.find_one({"_id": utilizator_id})
            if not utilizator:
                raise HTTPException(status_code=401, detail="Utilizator inexistent sau a fost sters din baza de date")
            return utilizator
    except jwt.ExpiredSignatureError:
        raise HTTPException(status_code=401, detail="Token expirat")
    except jwt.PyJWTError:
         raise HTTPException(status_code=401, detail="Token invalid")



@app.post("/upload/")
async def incarcare_imagini(imagine_front : UploadFile = File(...), imagine_profil: UploadFile = File(...), utilizator = Depends(get_utilizator_curent), db = Depends(get_db)): #imagine_incarcata este numele parametrului venit din frontend din evaluare_statica.component.ts
    global dict_distante_front
    global dict_distante_profil
    #UPLOAD_FOLDER = os.getcwd()
    #PROCESSED_FOLDER = "processed"
    #os.makedirs(UPLOAD_FOLDER, exist_ok=True)
    #os.makedirs(PROCESSED_FOLDER, exist_ok=True)
    
    #img_path = f"{UPLOAD_FOLDER}/{imagine_incarcata.filename}"
    #output_path = f"{PROCESSED_FOLDER}/processed_{imagine_incarcata.filename}"

    data_img_front = await imagine_front.read()
    data_img_profil = await imagine_profil.read()

    nparr_front = np.frombuffer(data_img_front, np.uint8)  # Convertește datele binare într-un array NumPy
    nparr_profil = np.frombuffer(data_img_profil, np.uint8)
    
    img_cv_front = cv2.imdecode(nparr_front, cv2.IMREAD_COLOR)
    img_cv_profil = cv2.imdecode(nparr_profil, cv2.IMREAD_COLOR)
        #with open(img_path, "wb") as f:
            #f.write(await file.read())
    
    distante["front"], img_prelucrata_front = prelucrare_img(img_cv_front, 1)
    distante["profil"], img_prelucrata_profil = prelucrare_img(img_cv_profil, 2)
    date_extrase_postura = preprocesare_date_img(distante)
    
    _, buffer_front = cv2.imencode('.jpg', img_prelucrata_front)
    img_front_b64 = base64.b64encode(buffer_front).decode("utf-8")

    _, buffer_profil = cv2.imencode('.jpg', img_prelucrata_profil)
    img_profil_b64 = base64.b64encode(buffer_profil).decode("utf-8")

    
    test_nou = {
        "utilizator_id": str(utilizator["_id"]),
        "rezultate" : date_extrase_postura,
        "imagine_front": img_front_b64,
        "imagine_profil": img_profil_b64,
        "datetime_test" : datetime.utcnow()
    }

    colectie_teste_postura = db["teste_postura"]
    colectie_teste_postura.insert_one(test_nou)

        #return {"processed_image": f"/processed/processed_{imagine_incarcata.filename}"}
@app.get("/test/{test_id}")
async def get_test_postura(test_id : str, utilizator=Depends(get_utilizator_curent), db=Depends(get_db)):

    colectie_teste_postura = db["teste_postura"]
    if not ObjectId.is_valid(test_id):
        raise HTTPException(status_code=400, detail="test_id invalid")
    else:
        test_postura = colectie_teste_postura.find_one({"_id" : ObjectId(test_id)})
        test_postura["_id"] = str(test_postura["_id"])
        if not test_postura:
            raise HTTPException(status_code=404, detail="Testul nu a putut fii gasit in baza de date")
            
    return test_postura

@app.get("/id_ultimul_test/")
async def get_id_ultimul_test(utilizator=Depends(get_utilizator_curent), db=Depends(get_db)):
    print("merge")
    colectie_teste_postura = db["teste_postura"]
    ultimul_test = colectie_teste_postura.find_one({"utilizator_id" : str(utilizator["_id"])}, sort=[("datetime_test", -1)])
    print(str(ultimul_test["_id"]))
    if not ultimul_test:
        raise HTTPException(status_code=404, detail="Nu a putut fi gasit ultimul test")

    return {"utimul_test_id": str(ultimul_test["_id"])}

In [10]:
@app.post("/creeaza_cont/")
async def creeaza_cont(utilizator_nou : Utilizator, db = Depends(get_db)):
    colectie_utilizatori = db["utilizatori"]
    
    if colectie_utilizatori.find_one({"email": utilizator_nou.email}):
        raise HTTPException(status = 400, cauza = "Exista deja un cont pe acest mail")
    elif colectie_utilizatori.find_one({"username": utilizator_nou.username}):
        raise HTTPException(status = 400, cauza = "Username existent")
    else:    
        try:
            pwd_context = CryptContext(schemes=["argon2"], deprecated="auto")
            parola_hashed = pwd_context.hash(utilizator_nou.parola)
            colectie_utilizatori.insert_one(
                {
                    "email" : utilizator_nou.email, 
                    "parola" : parola_hashed,
                    "username" : utilizator_nou.username
                }
            )
        except Exception as e:
            raise HTTPException(status_code = 400, detail = f"Contul nu a putut fii adaugat in baza de date:{str(e)}")



@app.post("/logare_cont/")
async def logare_cont(utilizator_existent : DateLogare, db = Depends(get_db)):
    colectie_utilizatori = db["utilizatori"]
    
    pwd_context = CryptContext(schemes=["argon2"], deprecated="auto")
    utilizator = colectie_utilizatori.find_one({"username" : utilizator_existent.username})
    
    if utilizator is None or not pwd_context.verify(utilizator_existent.parola, utilizator["parola"]):
        print(utilizator["parola"])
        print(utilizator["username"])
        raise HTTPException(status_code = 400, detail = "Email sau parola gresita. Incercati din nou")

    cheie_acces = generare_token_acces({"utilizator_id": str(utilizator["_id"])})
    return {
        "access_token": cheie_acces,
        "tip_token": "bearer",
        "mesaj_succes": "Autentificare realizata cu succes!"
    }
        

In [11]:
@app.get("/distante_postura_statica/")
async def get_distante():
    global distante

    date_extrase_postura = preprocesare_date_img(distante)
    if not all(distante.values()):
        return JSONResponse(content = {"error": "Imaginile nu au fost încă procesate complet"}, status_code = 400)
    else:
        date_extrase_postura = preprocesare_date_img(distante)
    return JSONResponse(content = date_extrase_postura)

In [12]:
 @app.get("/processed/{filename}")
async def get_imagine_prelucrata(filename: str):
    return FileResponse(f"{PROCESSED_FOLDER}/{filename}")

In [13]:
def adaug_text(img, coord_y, denumire, unghi):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype("arial.ttf", 20 * factor_scalare)
    
    box = draw.textbbox((0, 0), denumire, font)
    lungime_text = box[2] - box[0]
    inaltime_text = box[3] - box[1]
    draw.rectangle([(8, coord_y - inaltime_text - 16 * factor_scalare), (8 + lungime_text + 2 * factor_scalare, coord_y - 8 * factor_scalare)], (255, 255, 255))
    draw.text((10, coord_y - 2 *inaltime_text), denumire, (0, 0, 0), font)

    box = draw.textbbox((0, 0), unghi, font)
    lungime_text = box[2] - box[0]
    inaltime_text = box[3] - box[1]
    draw.rectangle([(img.shape[1] - 8 - lungime_text, coord_y - inaltime_text - 12 * factor_scalare), (img.shape[1] - 8, coord_y - 8 * factor_scalare)], (255, 255, 255))
    draw.text((img.shape[1] - 6 - lungime_text, coord_y - 2*inaltime_text), unghi, (0, 0, 0), font)

    img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    return img

In [14]:
def adaug_linii_suport_front(img):
    global mijloace
    global vect_unghiuri_front
    distante_y = []
    try:
            puncte = pose_model.pose_landmarks.landmark
            puncte_metri = pose_model.pose_world_landmarks.landmark
            
    except:
            pass
   
    print(len(puncte_metri))
    puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE]
    puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]
    denumiri = ["Urechi", "Umeri", "Solduri", "Genunchi", "Glezne"]
    mijloace = []
    vect_unghiuri_front = []

    coord_sold_dr_metri = (puncte_metri[mp_pose.PoseLandmark.RIGHT_HIP.value].x, puncte_metri[mp_pose.PoseLandmark.RIGHT_HIP.value].y, puncte_metri[mp_pose.PoseLandmark.RIGHT_HIP.value].z)
    diferente = masuratori_antropometrice(puncte_metri, puncte_linii_suport_dr, puncte_linii_suport_st)
    
    for (puncte_cheie_dr, puncte_cheie_st, denumire) in zip(puncte_linii_suport_dr, puncte_linii_suport_st, denumiri) :
        punct11 = (int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]), int(puncte[puncte_cheie_dr.value].z))
        punct12 = (int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]), int(puncte[puncte_cheie_st.value].z))

            
        cv2.line(img, punct11[:-1], punct12[:-1], (255, 255, 255), factor_scalare)
        cv2.circle(img, punct11[:-1], 10, (0, 0, 0), factor_scalare)
        cv2.circle(img, punct12[:-1], 10, (0, 0, 0), factor_scalare)
        coord_y = int(((puncte[puncte_cheie_dr.value].y + puncte[puncte_cheie_st.value].y)/2) * img.shape[0])
        coord_x = int(((puncte[puncte_cheie_dr.value].x + puncte[puncte_cheie_st.value].x)/2) * img.shape[1])
        mijloace.append((coord_x, coord_y));
        punct1 = (0, coord_y)
        punct2 = (img.shape[1], coord_y)
        cv2.line(img, punct1, punct2, (123, 45, 200), factor_scalare)
        unghi = round(calcul_unghi_orizont(punct11, punct12), 2)
        vect_unghiuri_front.append(unghi)
        unghi_str = str(unghi) + "°"
        img = adaug_text(img, coord_y, denumire, unghi_str)
       
        
    copie = img.copy()
    for i, puncte_cheie_dr in enumerate(puncte_linii_suport_dr[1:-1], 1):
            punct11 = (int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]))
            puncte_cheie_next = puncte_linii_suport_dr[i + 1]
            punct12 = (int(puncte[puncte_cheie_next.value].x * img.shape[1]), int(puncte[puncte_cheie_next.value].y * img.shape[0]))
            cv2.line(copie, punct11, punct12, (255, 255, 255), 2 * factor_scalare)
          
    for i, puncte_cheie_st in enumerate(puncte_linii_suport_st[1:-1], 1):
            punct11 = (int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]))
            puncte_cheie_next = puncte_linii_suport_st[i + 1]
            punct12 = (int(puncte[puncte_cheie_next.value].x * img.shape[1]), int(puncte[puncte_cheie_next.value].y * img.shape[0]))
            cv2.line(copie, punct11, punct12, (255, 255, 255), 2 * factor_scalare)

    img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)

    return img, diferente

In [15]:
def calcul_scalare_metrii_pixeli(coord_sold_dr_metri, coord_sold_dr, coord_sold_st):
    coord_x_mijloc = int(coord_sold_dr[0] + coord_sold_st[0])/2
    coord_y_mijloc = int(coord_sold_dr[1] + coord_sold_dr[1])/2
    distanta_pixeli = math.sqrt((coord_x_mijloc - coord_sold_dr[0]) ** 2 + (coord_y_mijloc - coord_sold_dr[1]) ** 2)

    distanta_metri = math.sqrt( coord_sold_dr_metri[0] ** 2 +  coord_sold_dr_metri[1]** 2)
    scalare = distanta_metri / distanta_pixeli

    return scalare
    # SA NU UIT sa schimb indexarea cu [4] ca arata ca dracu

In [16]:
def calcul_scalare_metrii_pixeli_profil(coord_sold_dr, coord_y_genunchi_m, coord_pct_suport_dr):
    #Presupun ca coordonatatele x si y ale sold_dr sunt aceleasi cu mijlocul intre solduri
    distanta_pixeli = math.sqrt((coord_pct_suport_dr[4][0] - coord_pct_suport_dr[3][0]) ** 2 + (coord_pct_suport_dr[4][1] - coord_pct_suport_dr[3][1]) ** 2)
    distanta_metri = math.sqrt(( coord_y_genunchi_m[0] - coord_sold_dr[0])** 2 +  (coord_y_genunchi_m[1] - coord_sold_dr[1])** 2)
    scalare = distanta_metri / distanta_pixeli

    return scalare

In [17]:
def masuratori_antropometrice(puncte_metri, puncte_linii_suport_dr, puncte_linii_suport_st):
    masuratori = []
    coord_pct_suport_dr = []
    coord_pct_suport_st = []
    for (puncte_cheie_dr, puncte_cheie_st) in zip(puncte_linii_suport_dr, puncte_linii_suport_st):
            coord_pct_suport_dr.append((puncte_metri[puncte_cheie_dr.value].x, puncte_metri[puncte_cheie_dr.value].y, puncte_metri[puncte_cheie_dr.value].z))
            coord_pct_suport_st.append((puncte_metri[puncte_cheie_st.value].x, puncte_metri[puncte_cheie_st.value].y, puncte_metri[puncte_cheie_st.value].z))
            
    for i, _ in enumerate(puncte_linii_suport_dr):
        masuratori.append(round(100 * (coord_pct_suport_st[i][1] - coord_pct_suport_dr[i][1]), 2))

    return masuratori

In [18]:
def calc_distanta(pct1, pct2):
    dist = math.sqrt((pct1[0] - pct2[0]) ** 2 + (pct1[1] - pct2[1]) ** 2 + (pct1[2] - pct2[2]) ** 2)
    return dist

In [19]:
def afisare_aliniament(img, coord, tip):
    copie_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(copie_pil)
    
    (_, y_urechi) = coord[0]
    (x_glezne, _) = coord[4]
        
    for i, (x, y) in enumerate(coord[:-1]):
        urm_x, urm_y = coord[i+1]
        pct_filler = [(x, y), (urm_x, urm_y), (x_glezne, urm_y), (x_glezne, y)]
        if (x > x_glezne and urm_x < x_glezne) or (x < x_glezne and urm_x > x_glezne):
            y_int = intersectie(x_glezne, y, x_glezne, urm_y, x, y, urm_x, urm_y)
            draw.polygon([(x, y), (x_glezne, y_int), (x_glezne, y)], (255, 0, 0))
            draw.polygon([(urm_x, urm_y), (x_glezne, y_int), (x_glezne, urm_y)], (255, 0, 0))
        else:
            draw.polygon(pct_filler, (255, 0, 0))

        if tip == 1:
            cv2.line(img, (x, y), (urm_x, urm_y), (255, 0, 0), factor_scalare)
            cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)
    if tip == 1:
        x, y = coord[-1]
        cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)
    copie = cv2.cvtColor(np.array(copie_pil), cv2.COLOR_RGB2BGR)
    img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)
    cv2.line(img, coord[4], (x_glezne, y_urechi), (0, 255, 0), factor_scalare)
    return img

In [20]:
def afisare_aliniament1(img, coord, tip):
    copie_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(copie_pil)
    distante = []
    (x_urechi, _) = coord[0]
    (_, y_glezne) = coord[4]
        
    for i, (x, y) in enumerate(coord[:-1]):
        urm_x, urm_y = coord[i+1]
        pct_filler = [(x, y), (urm_x, urm_y), (x_urechi, urm_y), (x_urechi, y)]
        if (x > x_urechi and urm_x < x_urechi) or (x < x_urechi and urm_x > x_urechi):
            y_int = intersectie(x_urechi, y, x_urechi, urm_y, x, y, urm_x, urm_y)
            draw.polygon([(x, y), (x_urechi, y_int), (x_urechi, y)], (255, 0, 0))
            draw.polygon([(urm_x, urm_y), (x_urechi, y_int), (x_urechi, urm_y)], (255, 0, 0))
        else:
            draw.polygon(pct_filler, (255, 0, 0))
            
        if tip == 2:
            cv2.line(img, (x, y), (x_urechi, y), (255, 255, 0), factor_scalare)
            distante.append(abs(x - x_urechi))
        
        if tip == 1:
            cv2.line(img, (x, y), (urm_x, urm_y), (255, 0, 0), factor_scalare)
            cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)

    if tip == 2:
        cv2.line(img, coord[-1], (x_urechi, coord[-1][1]), (255, 255, 0), factor_scalare)
        distante.append(abs(coord[-1][0] - x_urechi))
    if tip == 1:
        x, y = coord[-1]
        cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)
    copie = cv2.cvtColor(np.array(copie_pil), cv2.COLOR_RGB2BGR)
    img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)
    cv2.line(img, coord[0], (x_urechi, y_glezne), (0, 255, 0), factor_scalare)
    print(distante)
    return img, distante

In [21]:
def intersectie(x11, y11, x12, y12, x21, y21, x22, y22):
    y_int = int(((x11 * y12 - y11 * x12) * (y21 - y22) - (y11 - y12) * (x21 * y22 - y21 * x22)) / ((x11 - x12) * (y21 - y22) - (y11 - y12) * (x21 - x22)))
    return y_int

In [22]:
def afla_orientare(puncte):
    sold_stg = puncte.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].z
    sold_drept = puncte.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].z

    if sold_stg > 0:
        return "stanga"
    return "dreapta"
    

In [23]:
def adaug_linii_suport_profil(img):
    global puncte_de_interes
    global vect_unghiuri_profil
    try:
            puncte = pose_model.pose_landmarks
            puncte_metrii = pose_model.pose_world_landmarks.landmark
    except:
            pass


    puncte_de_interes = []
    vect_unghiuri_profil = []
    denumiri = ["Flexia cervicală", "Flexia șoldului", "Flexia genunchiului"]
    orientare = afla_orientare(puncte)
    if orientare == "dreapta":
        puncte_linii_suport = [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]
    else:
        puncte_linii_suport = [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE]
    print(orientare)

    coord_y_genunchi_m = (puncte_metrii[puncte_linii_suport[3].value].x, puncte_metrii[puncte_linii_suport[3].value].y, puncte_metrii[puncte_linii_suport[3].value].z)
    coord_sold_m = (puncte_metrii[puncte_linii_suport[2].value].x, puncte_metrii[puncte_linii_suport[2].value].y, puncte_metrii[puncte_linii_suport[2].value].z)
    copie = img.copy()
    for i, punct in enumerate(puncte_linii_suport[:-1]):
        coord_pct = (int(puncte.landmark[punct.value].x * img.shape[1]), int(puncte.landmark[punct.value].y * img.shape[0]), int(puncte.landmark[punct.value].z))
        if i == 0:
            if orientare == "dreapta":
                coord_pct = (int(puncte.landmark[punct.value].x * img.shape[1] + 15), int(puncte.landmark[punct.value].y * img.shape[0]), int(puncte.landmark[punct.value].z))
            else:
                coord_pct = (int(puncte.landmark[punct.value].x * img.shape[1] - 15), int(puncte.landmark[punct.value].y * img.shape[0]), int(puncte.landmark[punct.value].z))
        punct_urm = puncte_linii_suport[i+1]
        coord_pct_urm = (int(puncte.landmark[punct_urm.value].x * img.shape[1]), int(puncte.landmark[punct_urm.value].y * img.shape[0]), int(puncte.landmark[punct_urm.value].z))
        cv2.line(img, coord_pct[:-1], coord_pct_urm[:-1], (255, 255, 255), factor_scalare)
        cv2.circle(img, coord_pct[:-1], 10, (0, 0, 0), factor_scalare)

        if i:
            (_, coord_y, _) = coord_pct
            punct1 = (0, coord_y)
            punct2 = (img.shape[1], coord_y)
            cv2.line(img, punct1, punct2, (123, 45, 200), factor_scalare)
        
        puncte_de_interes.append(coord_pct[:-1])

    puncte_de_interes.append(coord_pct_urm[:-1])
    cv2.circle(img, coord_pct_urm[:-1], 10, (0, 0, 0), factor_scalare)
    scalare = calcul_scalare_metrii_pixeli_profil(coord_sold_m, coord_y_genunchi_m, puncte_de_interes)
    
    for denumire, (i, _) in zip(denumiri, enumerate(puncte_de_interes[1:4])):
        pct1 = puncte_de_interes[i]
        pct2 = puncte_de_interes[i+1]
        pct3 = puncte_de_interes[i + 2]
        unghi = round(180 - calcul_unghi_2_drepte(pct1, pct2, pct2, pct3), 2) #calculeaza unghiul dintre dreptele formate de (pct1, pct2) cu (pct2, pct3)  
        vect_unghiuri_profil.append(unghi)

        unghi_str = str(unghi) + "°"
        (_, coord_y) = pct2
        img = adaug_text(img, coord_y, denumire, unghi_str)

    return img, scalare
         
         

In [24]:
def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

INFO:     Started server process [10472]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56372 - "OPTIONS /logare_cont/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:56372 - "POST /logare_cont/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:57787 - "OPTIONS /test/67ee76325894964bdaf7adcd HTTP/1.1" 200 OK
67eaaf199f8cd973d6ee2854
INFO:     127.0.0.1:57787 - "GET /test/67ee76325894964bdaf7adcd HTTP/1.1" 200 OK
INFO:     127.0.0.1:57787 - "OPTIONS /test/67ee79e75894964bdaf7adcf HTTP/1.1" 200 OK
67eaaf199f8cd973d6ee2854
INFO:     127.0.0.1:57787 - "GET /test/67ee79e75894964bdaf7adcf HTTP/1.1" 200 OK
INFO:     127.0.0.1:57806 - "OPTIONS /test/67ee7c83002ae2cea1efbc42 HTTP/1.1" 200 OK
67eaaf199f8cd973d6ee2854
INFO:     127.0.0.1:57806 - "GET /test/67ee7c83002ae2cea1efbc42 HTTP/1.1" 200 OK
INFO:     127.0.0.1:57806 - "OPTIONS /test/67ee7fa710a994e635c5c7ef HTTP/1.1" 200 OK
67eaaf199f8cd973d6ee2854
INFO:     127.0.0.1:57806 - "GET /test/67ee7fa710a994e635c5c7ef HTTP/1.1" 200 OK
